In [1]:
#BiocManager::install("edgeR")
library(edgeR)
library(sva)

Warning message:
"package 'edgeR' was built under R version 3.6.2"Loading required package: limma
Warning message:
"package 'limma' was built under R version 3.6.2"Loading required package: mgcv
Loading required package: nlme
This is mgcv 1.8-28. For overview type 'help("mgcv-package")'.
Loading required package: genefilter
Loading required package: BiocParallel
Warning message:
"package 'BiocParallel' was built under R version 3.6.2"

#  GeTMM (Gene length corrected TMM) from [Smid et al.](https://doi.org/10.1186/s12859-018-2246-7)(2018)

In [2]:
x <- read.csv("./output_data/gexdata.csv", row.names = 1, header= TRUE, check.names = FALSE)
# the check names argument will not change the name of the columns

In [3]:
head(x)

,SJHBBBM4X52,SJHBBBM4JQZ,SJHBBBM5SGJ,SJHBBBM5TXY,SJHBBBM2GV5,SJHBBBM39BT,SJHBBBM67H6,SJHBBBM2BBR,SJHBBBN9RBJ,SJHBBBM4PX3,SJHBBBN9RDM,SJHBBBN9R9H,SJHBBBM588V,SJHBBBN9RFN,SJHBBBM55F5,SJHBBBM5GW9,SJHBBBM5PY5,SJHBBBM52XR,SJHBBBM5YWS
ENSG00000186092,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSG00000284733,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSG00000284662,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSG00000187634,3,0,0,6,9,1,24,2,24,2,3,2,3,5,11,0,7,3,1
ENSG00000188976,3785,3524,3861,2710,11324,4393,4119,8499,3053,2708,1970,3239,1506,3098,2188,4012,1707,5557,2273
ENSG00000187961,756,495,193,898,2269,883,934,1311,514,412,336,542,238,329,188,871,196,1225,400


In [4]:
dim(x)

[1] 19774    19

In [5]:
batches <- read.table("./output_data/batches.txt", header = FALSE, sep = "")

In [6]:
batches$V1

[1] batch1-GEO batch1-GEO batch1-GEO batch1-GEO batch1-GEO batch1-GEO
 [7] batch1-GEO batch1-GEO batch1-GEO batch1-GEO batch1-GEO batch1-GEO
[13] batch1-GEO batch1-GEO batch1-GEO batch1-GEO batch1-GEO batch1-GEO
[19] batch1-GEO
Levels: batch1-GEO

In [7]:
group = batches$V1

In [8]:
genes <- read.csv("./data/finalgenelength.csv", row.names = 1, header= TRUE, check.names = FALSE)
head(genes)

,length
ENSG00000186092,2618
ENSG00000284733,939
ENSG00000284662,995
ENSG00000187634,4172
ENSG00000188976,5540
ENSG00000187961,3402


In [9]:
dim(genes)

[1] 19774     1

In [10]:
sum(rownames(genes) == rownames(x))

[1] 19774

In [11]:
head(x)

,SJHBBBM4X52,SJHBBBM4JQZ,SJHBBBM5SGJ,SJHBBBM5TXY,SJHBBBM2GV5,SJHBBBM39BT,SJHBBBM67H6,SJHBBBM2BBR,SJHBBBN9RBJ,SJHBBBM4PX3,SJHBBBN9RDM,SJHBBBN9R9H,SJHBBBM588V,SJHBBBN9RFN,SJHBBBM55F5,SJHBBBM5GW9,SJHBBBM5PY5,SJHBBBM52XR,SJHBBBM5YWS
ENSG00000186092,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSG00000284733,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSG00000284662,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSG00000187634,3,0,0,6,9,1,24,2,24,2,3,2,3,5,11,0,7,3,1
ENSG00000188976,3785,3524,3861,2710,11324,4393,4119,8499,3053,2708,1970,3239,1506,3098,2188,4012,1707,5557,2273
ENSG00000187961,756,495,193,898,2269,883,934,1311,514,412,336,542,238,329,188,871,196,1225,400


#### gene length in kb (x/(lenght/1000))

In [12]:
data <- ( (x*10^3 )/genes$length)
head(data)

,SJHBBBM4X52,SJHBBBM4JQZ,SJHBBBM5SGJ,SJHBBBM5TXY,SJHBBBM2GV5,SJHBBBM39BT,SJHBBBM67H6,SJHBBBM2BBR,SJHBBBN9RBJ,SJHBBBM4PX3,SJHBBBN9RDM,SJHBBBN9R9H,SJHBBBM588V,SJHBBBN9RFN,SJHBBBM55F5,SJHBBBM5GW9,SJHBBBM5PY5,SJHBBBM52XR,SJHBBBM5YWS
ENSG00000186092,0.0000000,0.0000,0.00000,0.000000,0.000000,0.0000000,0.000000,0.0000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.0000,0.000000,0.0000000,0.0000000
ENSG00000284733,0.0000000,0.0000,0.00000,0.000000,0.000000,0.0000000,0.000000,0.0000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.0000,0.000000,0.0000000,0.0000000
ENSG00000284662,0.0000000,0.0000,0.00000,0.000000,0.000000,0.0000000,0.000000,0.0000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.000000,0.0000,0.000000,0.0000000,0.0000000
ENSG00000187634,0.7190796,0.0000,0.00000,1.438159,2.157239,0.2396932,5.752637,0.4793864,5.752637,0.4793864,0.7190796,0.4793864,0.7190796,1.198466,2.636625,0.0000,1.677852,0.7190796,0.2396932
ENSG00000188976,683.2129964,636.1011,696.93141,489.169675,2044.043321,792.9602888,743.501805,1534.1155235,551.083032,488.8086643,355.5956679,584.6570397,271.8411552,559.205776,394.945848,724.1877,308.122744,1003.0685921,410.2888087
ENSG00000187961,222.2222222,145.5026,56.73133,263.962375,666.960611,259.5532040,274.544386,385.3615520,151.087596,121.1052322,98.7654321,159.3180482,69.9588477,96.707819,55.261611,256.0259,57.613169,360.0823045,117.5778954


In [13]:
d <- DGEList(counts=data, group=group)

In [14]:
head(d)

An object of class "DGEList"
$counts
                SJHBBBM4X52 SJHBBBM4JQZ SJHBBBM5SGJ SJHBBBM5TXY SJHBBBM2GV5
ENSG00000186092   0.0000000      0.0000     0.00000    0.000000    0.000000
ENSG00000284733   0.0000000      0.0000     0.00000    0.000000    0.000000
ENSG00000284662   0.0000000      0.0000     0.00000    0.000000    0.000000
ENSG00000187634   0.7190796      0.0000     0.00000    1.438159    2.157239
ENSG00000188976 683.2129964    636.1011   696.93141  489.169675 2044.043321
ENSG00000187961 222.2222222    145.5026    56.73133  263.962375  666.960611
                SJHBBBM39BT SJHBBBM67H6  SJHBBBM2BBR SJHBBBN9RBJ SJHBBBM4PX3
ENSG00000186092   0.0000000    0.000000    0.0000000    0.000000   0.0000000
ENSG00000284733   0.0000000    0.000000    0.0000000    0.000000   0.0000000
ENSG00000284662   0.0000000    0.000000    0.0000000    0.000000   0.0000000
ENSG00000187634   0.2396932    5.752637    0.4793864    5.752637   0.4793864
ENSG00000188976 792.9602888  743.501805 1534.1155235  551.083032 488.8086643
ENSG00000187961 259.5532040  274.544386  385.3615520  151.087596 121.1052322
                SJHBBBN9RDM SJHBBBN9R9H SJHBBBM588V SJHBBBN9RFN SJHBBBM55F5
ENSG00000186092   0.0000000   0.0000000   0.0000000    0.000000    0.000000
ENSG00000284733   0.0000000   0.0000000   0.0000000    0.000000    0.000000
ENSG00000284662   0.0000000   0.0000000   0.0000000    0.000000    0.000000
ENSG00000187634   0.7190796   0.4793864   0.7190796    1.198466    2.636625
ENSG00000188976 355.5956679 584.6570397 271.8411552  559.205776  394.945848
ENSG00000187961  98.7654321 159.3180482  69.9588477   96.707819   55.261611
                SJHBBBM5GW9 SJHBBBM5PY5  SJHBBBM52XR SJHBBBM5YWS
ENSG00000186092      0.0000    0.000000    0.0000000   0.0000000
ENSG00000284733      0.0000    0.000000    0.0000000   0.0000000
ENSG00000284662      0.0000    0.000000    0.0000000   0.0000000
ENSG00000187634      0.0000    1.677852    0.7190796   0.2396932
ENSG00000188976    724.1877  308.122744 1003.0685921 410.2888087
ENSG00000187961    256.0259   57.613169  360.0823045 117.5778954

$samples
                 group lib.size norm.factors
SJHBBBM4X52 batch1-GEO  8188287            1
SJHBBBM4JQZ batch1-GEO  8525097            1
SJHBBBM5SGJ batch1-GEO  8318486            1
SJHBBBM5TXY batch1-GEO  8634618            1
SJHBBBM2GV5 batch1-GEO 16643999            1
14 more rows ...

In [15]:
TMM <- calcNormFactors(d, method="TMM")

In [16]:
head(TMM)

An object of class "DGEList"
$counts
                SJHBBBM4X52 SJHBBBM4JQZ SJHBBBM5SGJ SJHBBBM5TXY SJHBBBM2GV5
ENSG00000186092   0.0000000      0.0000     0.00000    0.000000    0.000000
ENSG00000284733   0.0000000      0.0000     0.00000    0.000000    0.000000
ENSG00000284662   0.0000000      0.0000     0.00000    0.000000    0.000000
ENSG00000187634   0.7190796      0.0000     0.00000    1.438159    2.157239
ENSG00000188976 683.2129964    636.1011   696.93141  489.169675 2044.043321
ENSG00000187961 222.2222222    145.5026    56.73133  263.962375  666.960611
                SJHBBBM39BT SJHBBBM67H6  SJHBBBM2BBR SJHBBBN9RBJ SJHBBBM4PX3
ENSG00000186092   0.0000000    0.000000    0.0000000    0.000000   0.0000000
ENSG00000284733   0.0000000    0.000000    0.0000000    0.000000   0.0000000
ENSG00000284662   0.0000000    0.000000    0.0000000    0.000000   0.0000000
ENSG00000187634   0.2396932    5.752637    0.4793864    5.752637   0.4793864
ENSG00000188976 792.9602888  743.501805 1534.1155235  551.083032 488.8086643
ENSG00000187961 259.5532040  274.544386  385.3615520  151.087596 121.1052322
                SJHBBBN9RDM SJHBBBN9R9H SJHBBBM588V SJHBBBN9RFN SJHBBBM55F5
ENSG00000186092   0.0000000   0.0000000   0.0000000    0.000000    0.000000
ENSG00000284733   0.0000000   0.0000000   0.0000000    0.000000    0.000000
ENSG00000284662   0.0000000   0.0000000   0.0000000    0.000000    0.000000
ENSG00000187634   0.7190796   0.4793864   0.7190796    1.198466    2.636625
ENSG00000188976 355.5956679 584.6570397 271.8411552  559.205776  394.945848
ENSG00000187961  98.7654321 159.3180482  69.9588477   96.707819   55.261611
                SJHBBBM5GW9 SJHBBBM5PY5  SJHBBBM52XR SJHBBBM5YWS
ENSG00000186092      0.0000    0.000000    0.0000000   0.0000000
ENSG00000284733      0.0000    0.000000    0.0000000   0.0000000
ENSG00000284662      0.0000    0.000000    0.0000000   0.0000000
ENSG00000187634      0.0000    1.677852    0.7190796   0.2396932
ENSG00000188976    724.1877  308.122744 1003.0685921 410.2888087
ENSG00000187961    256.0259   57.613169  360.0823045 117.5778954

$samples
                 group lib.size norm.factors
SJHBBBM4X52 batch1-GEO  8188287     1.226914
SJHBBBM4JQZ batch1-GEO  8525097     1.184659
SJHBBBM5SGJ batch1-GEO  8318486     1.166990
SJHBBBM5TXY batch1-GEO  8634618     1.019168
SJHBBBM2GV5 batch1-GEO 16643999     1.048387
14 more rows ...

In [17]:
dim(TMM)

[1] 19774    19

In [18]:
CPM <- cpm(TMM, log = TRUE)
head(CPM)

,SJHBBBM4X52,SJHBBBM4JQZ,SJHBBBM5SGJ,SJHBBBM5TXY,SJHBBBM2GV5,SJHBBBM39BT,SJHBBBM67H6,SJHBBBM2BBR,SJHBBBN9RBJ,SJHBBBM4PX3,SJHBBBN9RDM,SJHBBBN9R9H,SJHBBBM588V,SJHBBBN9RFN,SJHBBBM55F5,SJHBBBM5GW9,SJHBBBM5PY5,SJHBBBM52XR,SJHBBBM5YWS
ENSG00000186092,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.0904160,-2.090416,-2.0904160,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416
ENSG00000284733,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.0904160,-2.090416,-2.0904160,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416
ENSG00000284662,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.0904160,-2.090416,-2.0904160,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416,-2.090416
ENSG00000187634,-1.706562,-2.090416,-2.090416,-1.328298,-1.480190,-1.949401,-0.3501239,-1.934003,0.1999545,-1.672060,-1.598543,-1.755570,-1.150391,-1.125532,-0.303728,-2.090416,-0.837277,-1.644218,-1.762831
ENSG00000188976,6.092568,5.982292,6.170470,5.802748,6.874997,6.321903,6.1554557,6.431027,6.4557950,6.335916,5.567127,6.229754,6.353578,6.707766,6.433239,6.353666,5.904418,6.894250,6.682187
ENSG00000187961,4.482497,3.872037,2.603794,4.917914,5.265192,4.719361,4.7262540,4.449561,4.5990917,4.335585,3.737429,4.366075,4.407298,4.191508,3.619831,4.861147,3.509768,5.421305,4.887353


In [19]:
min(CPM)


[1] -2.090416

In [20]:
max(CPM)

[1] 16.96304

In [21]:
write.csv(CPM, './output_data/gex-GeTMM.csv')